In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from detection import train_detector, detect

train_dir = '00_input/train'
im_size = 100
coords_size = 28

/home/g5sokolovroman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from keras.models import Model
from keras import backend as K
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint
from keras.utils import Sequence

from keras.layers import (Input, concatenate, Conv2D, MaxPooling2D, 
                          UpSampling2D, Convolution2D, ZeroPadding2D, 
                          BatchNormalization, Activation, concatenate, 
                          Flatten, Dense, merge)
from keras.optimizers import rmsprop

def get_model():
    inputs = Input(shape=(im_size, im_size, 1))
    conv = Conv2D(filters=256, kernel_size=(3,3), padding='same')(inputs)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=128, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    flatten = Flatten()(relu)
    predictions = Dense(coords_size, activation='tanh')(flatten)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    return model

def first_model():
    inputs = Input(shape=(im_size, im_size, 1))
    conv = Conv2D(filters=256, kernel_size=(3,3), padding='same')(inputs)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=128, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    conv = Conv2D(filters=64, kernel_size=(3,3), padding='same')(relu)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    maxpool = MaxPooling2D()(relu)
    
    conv = Conv2D(filters=32, kernel_size=(3,3), padding='same')(maxpool)
    batchnorm = BatchNormalization()(conv)
    relu = Activation('relu')(batchnorm)
    
    flatten = Flatten()(relu)
    predictions = Dense(coords_size, activation='tanh')(flatten)
    
    model = Model(inputs=inputs, outputs=predictions)
    model.compile(optimizer='rmsprop', loss='mean_squared_error')
    return model

Using TensorFlow backend.


In [9]:
train_gt = read_csv(train_dir+'/gt.csv')
train_detector(train_gt, train_dir+'/images', False, first_model, 'first')

from keras.models import load_model
model = load_model('first_10_0.0039.hdf5')
ans_dict = detect(model, '00_input/test/images')